In [45]:
import skfuzzy
import numpy as np
from utils.datastore import *

In [44]:
ds = DataStore()
D = ds.build_datasets(labeled=False)

In [46]:
# Hiperparameters:
metric = 'cityblock'
ncenters = 7
niter = 50

best_results = []

for Di in D:
    results = []
    J = np.nan
    best_result_index = 0
    for i in range(niter):
        results.append(
            skfuzzy.cluster.cmeans(Di.transpose(), ncenters, 2, error=0.005, metric=metric, maxiter=1000, init=None)
        )
        jm = results[i][4][-1]
        if np.isnan(J):
            J = jm
        else:
            if jm < J:
                best_result_index = i
    best_results.append(results[best_result_index])

        